In [1]:
!nvidia-smi

Thu May  2 17:42:10 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:41:00.0 Off |                  Off |
| 30%   40C    P8             23W /  230W |    3136MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from torch.optim import Adam
from torchvision.utils import make_grid
from diffusers import AutoencoderKL
from accelerate import Accelerator, InitProcessGroupKwargs
from accelerate.utils import ProjectConfiguration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
#############################
im_channels = 3

# Create the model and dataset #
# params = {
#     "block_out_channels": [
#     128,
#     256,
#     512,
#     512
#   ],
#   "down_block_types": [
#     "DownEncoderBlock2D",
#     "DownEncoderBlock2D",
#     "DownEncoderBlock2D",
#     "DownEncoderBlock2D"
#   ],
#   "in_channels": 3,
#   "latent_channels": 4,
#   "layers_per_block": 2,
#   "out_channels": 3,
#   "sample_size": 32,
#   "scaling_factor": 0.18215,
#   "up_block_types": [
#     "UpDecoderBlock2D",
#     "UpDecoderBlock2D",
#     "UpDecoderBlock2D",
#     "UpDecoderBlock2D"
#   ]
# }

# model = AutoencoderKL(**params).to(device)
model = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae").to(device)

# model = AutoencoderKL(in_channels=im_channels, out_channels=im_channels).to(device)

/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/transformers/utils/generic.py:485: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/transformers/utils/generic.py:342: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/diffusers/utils/outputs.py:64: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/diffusers/utils/outputs.py:64: UserWarning: torch.utils._pytree._register_pytree_node is de

In [3]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from datasets import load_dataset

resolution = 256

augmentations = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

# augmentations = transforms.Compose(
#     [
#         transforms.Resize(resolution, interpolation=transforms.InterpolationMode.BILINEAR),
#         transforms.RandomCrop(resolution),
#         transforms.Lambda(lambda x: x),
#         transforms.ToTensor(),
#         transforms.Normalize([0.5], [0.5]),
#     ]
# )

def transform_images(examples):
    images = [augmentations(image.convert("RGB")) for image in examples["image"]]
    return {"input": images}

train_batch_size = 4
dataset_path = "ddpm-data/summer2winter_yosemite/train"
train_ds = load_dataset("imagefolder", data_dir=dataset_path, split="train")
print(f"Dataset size: {len(train_ds)}")
train_ds.set_transform(transform_images)
train_dataloader = torch.utils.data.DataLoader(
    train_ds, batch_size=train_batch_size, shuffle=True, num_workers=0
)

Resolving data files:   0%|          | 0/2193 [00:00<?, ?it/s]

Dataset size: 2193


In [4]:
# def forward(model, x):
#     # z_mu, z_var = Q(X)
#     # z = sample_z(z_mu, z_var)
#     # X_sample = P(z)

#     # # Loss
#     # recon_loss = nn.binary_cross_entropy(X_sample, X, size_average=False) / mb_size
#     # kl_loss = torch.mean(0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var, 1))
#     # loss = recon_loss + kl_loss

#     # posterior = model.encode(x).latent_dist
#     # print(posterior)
#     # if sample_posterior:
#     #     z = posterior.sample(generator=generator)
#     # else:
#     #     z = posterior.mode()
#     # dec = self.decode(z).sample

#     # if not return_dict:
#     #     return (dec,)

#     # return DecoderOutput(sample=dec)


#     posterior = model.encode(x).latent_dist
#     z = posterior.mode()
#     # z = posterior.sample()
    
#     x_sample = model.decode(z).sample
#     # Loss
#     recon_loss = F.mse_loss(x_sample, x)
#     # kl_loss = torch.mean(0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var, 1))
#     kl_loss = posterior.kl()
#     loss = recon_loss + kl_loss
#     return x_sample,  loss.mean(), recon_loss.mean(), kl_loss.mean()

In [5]:
# def forwardV2(model, x):
#     # z_mu, z_var = Q(X)
#     # z = sample_z(z_mu, z_var)
#     # X_sample = P(z)

#     # # Loss
#     # recon_loss = nn.binary_cross_entropy(X_sample, X, size_average=False) / mb_size
#     # kl_loss = torch.mean(0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var, 1))
#     # loss = recon_loss + kl_loss

#     # posterior = model.encode(x).latent_dist
#     # print(posterior)
#     # if sample_posterior:
#     #     z = posterior.sample(generator=generator)
#     # else:
#     #     z = posterior.mode()
#     # dec = self.decode(z).sample

#     # if not return_dict:
#     #     return (dec,)

#     # return DecoderOutput(sample=dec)


#     posterior = model.encode(x).latent_dist
#     z = posterior.mode()
#     # z = posterior.sample()
    
#     x_sample = model.decode(z).sample
#     # print(x_sample)
#     # Loss
#     # recon_loss = F.mse_loss(x_sample, x)
    
#     recon_loss = 0.5 * F.mse_loss(
#         x_sample.reshape(x.shape[0], -1),
#         x.reshape(x.shape[0], -1),
#         reduction="none",
#     ).sum(dim=-1)
#     # kl_loss = torch.mean(0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var, 1))
#     # kl_loss = posterior.kl()
#     # loss = recon_loss + kl_loss



#     # return 0.5 * torch.sum(
#     #                 torch.pow(self.mean, 2) + self.var - 1.0 - self.logvar,
#     #                 dim=[1, 2, 3],
#     #             )


#     # print(posterior.logvar.shape, posterior.mean.shape)
#     KLD = 0.5 * torch.sum(
#         torch.pow(posterior.mean, 2) + posterior.var - 1.0 - posterior.logvar,
#         dim=[1, 2, 3],
#     )

#     loss = (recon_loss + KLD).mean(dim=0)
#     return x_sample, loss , recon_loss.mean(dim=0), KLD.mean(dim=0)

In [6]:
# batch = next(iter(train_dataloader))

In [7]:
# out = forwardV2(model, batch["input"].to(device))
# out[1:]

In [8]:
# out[-1]

In [9]:
# import matplotlib.pyplot as plt

# _x = out[0][0].detach().cpu()
# plt.imshow(_x.permute(1, 2, 0))

In [10]:
# def show_img(img_tensor):
#     img_tensor = (img_tensor - img_tensor.min()) / (img_tensor.max() - img_tensor.min())
#     img_tensor = torch.clamp(img_tensor, 0, 1) * 255
    
#     # Convert to numpy and display using matplotlib
#     image = img_tensor.numpy().transpose(1, 2, 0)  # Assuming the format is CxHxW, convert to HxWxC
#     plt.imshow(image)
#     plt.axis('off')  # Turn off axis numbers and ticks
#     plt.show()
#     return img_tensor

In [11]:
# show_img(_x)

In [ ]:
def train_loop(train_dataloader):
    from tqdm.auto import tqdm, trange
    import os
    from diffusers.optimization import get_scheduler
    import lpips

    task_name = "summer-2-winter-vae-hf"
    num_epochs = 20
    gradient_accumulation_steps = 1

    model = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=1e-5,
        betas=(0.95, 0.999),
        weight_decay=1e-6,
        eps=1e-08,
    )


    lr_scheduler = get_scheduler(
        "cosine",
        optimizer=optimizer,
        num_warmup_steps=100 * gradient_accumulation_steps,
        num_training_steps=(len(train_dataloader) * num_epochs),
    )

    step_count = 0
    image_save_steps = 64
    acc_steps = 4
    img_save_count = 0
    vae_autoencoder_ckpt_name = 'vqvae_autoencoder_ckpt.pth'

    kl_scale = 1e-6
    lpips_scale = 1e-1
    lpips_loss_fn = lpips.LPIPS(net="alex").to(device)

    with trange(num_epochs, desc="Epoch") as ep:
        for epoch_idx in ep:
            losses = []
            mse_losses = []
            kl_losses = []
            lpips_losses = []
            optimizer.zero_grad()
            with tqdm(train_dataloader, desc="Train") as tr_loop:
                for batch in tr_loop:
                    step_count += 1
                    x = batch["input"].to(device)

                    posterior = model.encode(x).latent_dist
                    z = posterior.mode()
                    pred = model.decode(z).sample
                    kl_loss = posterior.kl().mean()
                    mse_loss = F.mse_loss(pred, x, reduction="mean")
                    lpips_loss = lpips_loss_fn(pred, x).mean()

                    loss = (
                        mse_loss + lpips_scale * lpips_loss + kl_scale * kl_loss
                    )
                    
                    # Image Saving Logic
                    if step_count % image_save_steps == 0 or step_count == 1:
                        sample_size = min(8, x.shape[0])
                        save_output = torch.clamp(pred[:sample_size], -1., 1.).detach().cpu()
                        save_output = ((save_output + 1) / 2)
                        save_input = ((x[:sample_size] + 1) / 2).detach().cpu()
                        
                        grid = make_grid(torch.cat([save_input, save_output], dim=0), nrow=sample_size)
                        img = transforms.ToPILImage()(grid)
                        if not os.path.exists(os.path.join(task_name,'vqvae_autoencoder_samples')):
                            os.mkdir(os.path.join(task_name, 'vqvae_autoencoder_samples'))
                        img.save(os.path.join(task_name,'vqvae_autoencoder_samples',
                                            'current_autoencoder_sample_{}.png'.format(img_save_count)))
                        img_save_count += 1
                        img.close()
                    tr_loop.set_postfix(loss=loss.item(), mse_loss=mse_loss.item(), kl_loss=kl_loss.item(), lpips_loss=lpips_loss.item())
                    losses.append(loss.item())
                    mse_losses.append(mse_loss.item())
                    kl_losses.append(kl_loss.item())
                    lpips_losses.append(lpips_loss.item())
                    loss.backward()
                
                    
                    if step_count % acc_steps == 0:
                        optimizer.step()
                        optimizer.zero_grad()
            optimizer.step()
            optimizer.zero_grad()
            avg_epoch_loss = sum(losses) / len(losses)
            avg_epoch_mse_loss = sum(mse_losses) / len(mse_losses)
            avg_epoch_kl_loss = sum(kl_losses) / len(kl_losses)
            avg_epoch_lpips_loss = sum(lpips_losses) / len(lpips_losses)
            ep.set_postfix(
                avg_epoch_loss=avg_epoch_loss, 
                avg_epoch_mse_loss=avg_epoch_mse_loss, 
                avg_epoch_kl_loss=avg_epoch_kl_loss,
                avg_epoch_lpips_loss=avg_epoch_lpips_loss
            )
            
            torch.save(model.state_dict(), os.path.join(task_name, vae_autoencoder_ckpt_name))
    print('Done Training...')
    

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /nfshomes/skarki/scratch/miniconda3/envs/llm/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Train:   0%|          | 0/549 [00:00<?, ?it/s]

KeyboardInterrupt: 